# 04 - Model Evaluation (Clustering)

## I. Preliminaries

Import the libraries and functions for model evaluation.

In [ ]:
# Libraries Here

Import the trained DGD model for evaluation.

In [ ]:
dgd_final = model.MetaboDGD(
    latent_dim=50,
    output_dim=np_normal_log.shape[1],
    dec_hidden_layers_dim=[500, 1500],
    dec_output_prediction_type='mean',
    dec_output_activation_type='softplus',
    n_comp=8,
    cm_type='diagonal',
    gaussian_stddev=0.5
)

dgd_final.dec.load_state_dict(torch.load('torch_outputs/torch_outputs_dec.pt'))
dgd_final.gmm.load_state_dict(torch.load('torch_outputs/torch_outputs_gmm.pt'))
# dgd_final.dec.load_state_dict(torch.load('98_85_acc/torch_outputs_dec.pt'))
# dgd_final.gmm.load_state_dict(torch.load('98_85_acc/torch_outputs_gmm.pt'))

Import the learned representations of the training samples.

In [ ]:
from metaboDGD.src.latent import RepresentationLayer
train_rep_final = RepresentationLayer(values=torch.zeros(size=(np_normal_log.shape[0], dgd_final.gmm.dim)))
train_rep_final.load_state_dict(torch.load('torch_outputs/torch_outputs_train_rep.pt'))
# train_rep_final.load_state_dict(torch.load('98_85_acc/torch_outputs_train_rep.pt'))

## II. Model Evaluation - Figure X

## III. Model Evaluation - Figure Y